In [71]:
import pandas as pd
import dask.dataframe as dd

pd.options.display.float_format = '{:.7f}'.format


#구독번호 리스트를 읽습니다.
#파일명은 read_csv('') 여기 작은따옴표 사이에 .csv 포맷까지 다 써주세요.
subList=pd.read_csv('sarah_subList.csv')['SubID'].values.tolist()

#여기 url='' 작은따옴표 사이에 사용량 파일 .csv 포맷까지 적어주세요.
url = 'Detail_BillingProfile_9ab0204c-fbfd-4078-b24a-0860302168ca_202302_en.csv'

usage = dd.read_csv(url, usecols=['customerTenantId','customerName','subscriptionId', 'product','pricingModel','frequency','costInBillingCurrency','date',
                                  'reservationId','reservationName', 'benefitId']
                    ,low_memory=False
                   ,dtype={'reservationId': 'object',
       'reservationName': 'object','costInBillingCurrency': 'float64',
       'subscriptionId': 'object','benefitId': 'object'})

filtered = usage.loc[usage["subscriptionId"].isin(subList)]
pd_usage = filtered.compute()

print("사용량 파일 필터링 끝 ---- 다음 아래 셀을 실행해주세요.")

사용량 파일 필터링 끝 ---- 다음 아래 셀을 실행해주세요.


In [72]:
####Total Usage######################################################################################################
summary = pd.pivot_table(pd_usage, index=["customerName","customerTenantId","subscriptionId","product"], 
                         values=["costInBillingCurrency"], aggfunc="sum", margins=True, margins_name="Total")
result = pd.DataFrame(summary.to_records())
display(summary)

TenantID = result["customerTenantId"].values[0]


costInBillingCurrency
customerName customerTenantId                     subscriptionId                       product                                                                  
에이디티캡스       33600eeb-f6e6-4207-ab2f-8dd2de8758e7 8d95d00a-c04f-4a20-8c79-da13a5fba4cc Azure CDN from Akamai - Standard - Zone 1                       0.0000000
                                                                                       Azure Monitor - Alerts                                          0.0000000
                                                                                       Azure Monitor - Emails                                          0.0000000
                                                                                       Backup - Azure VM - KR Central                              75772.0000000
                                                                                       Backup - Standard - KR Central                              68232.0000000
                                                                                       Bandwidth Inter-Region - Inter Continent - Inte...              0.0000000
                                                                                       Bandwidth Inter-Region - Intra Continent - Asia               874.0000000
                                                                                       Basic Application Gateway - Medium                              0.0000000
                                                                                       Blob Storage - Hot LRS - KR Central                             0.0000000
                                                                                       Files - LRS                                                     0.0000000
                                                                                       General Block Blob - LRS                                      519.0000000
                                                                                       General Block Blob - LRS - KR Central                         304.0000000
                                                                                       IP Addresses - Basic IPv4                                   23073.0000000
                                                                                       Insight and Analytics                                           0.0000000
                                                                                       Log Analytics - Pay-as-you-go                                 658.0000000
                                                                                       Microsoft Defender for SQL - Standard                       16155.0000000
                                                                                       Microsoft Defender for Servers - Standard P2               192282.0000000
                                                                                       Network Watcher                                                 0.0000000
                                                                                       Queues - LRS                                                    0.0000000
                                                                                       Rtn Preference: MGN - AU East                              194018.0000000
                                                                                       SQL Database Single Standard - S0 - KR Central              17764.0000000
                                                                                       SQL Database Single Standard - S3 - KR Central             178851.0000000
                                                                                       Standard Page Blob - LRS                                    84336.0000000
                                                                                       Standard Page Blob - LRS - 

In [73]:
####Calculate Usage refund###########################################################################################
TotalAmount = 0
refund = 0
usageFlag = True

sort_usage = pd_usage[pd_usage.pricingModel =='OnDemand']
if(sort_usage.empty):
    print("#### No Usage ####")
else:
    usage = pd.pivot_table(pd_usage, index=["customerName","customerTenantId","subscriptionId"], values=["costInBillingCurrency"], aggfunc="sum", 
                          margins=True, margins_name="Total")

    TotalAmount = usage.tail(n=1)['costInBillingCurrency']
    refund = TotalAmount*0.1
    usage.reset_index(inplace=True)
    usage.loc[len(usage.index)] = ['','','Refund(Total*10%)',refund.values[0]]
    display(usage)


,customerName,customerTenantId,subscriptionId,costInBillingCurrency
0,에이디티캡스,33600eeb-f6e6-4207-ab2f-8dd2de8758e7,8d95d00a-c04f-4a20-8c79-da13a5fba4cc,2500428.0000000
1,Total,,,2500428.0000000
2,,,Refund(Total*10%),250042.8000000


In [75]:
##Download
file_name = 'sarah_CSP.xlsx'
with pd.ExcelWriter(file_name) as writer:
    #total.to_excel(writer, sheet_name="Total Refund Amount", startrow=1, startcol=1)
    usage.to_excel(writer, sheet_name="Usage Refund Calc", startrow=1, startcol=1, index=False)
    #if(riFlag):
    #    purchase_ri.to_excel(writer, sheet_name="RI Refund Calc", startrow=1, startcol=1, index=False) 
    pd_usage.to_excel(writer, sheet_name="raw Usage Summary", startrow=0, startcol=0, index=False)
display("DONE!! Go to https://20.214.188.193:8000/tree/Refund to check " + file_name + " file")

'DONE!! Go to https://20.214.188.193:8000/tree/Refund to check sarah_CSP.xlsx file'